In [29]:
import requests
import json

BASE_ENDPOINT = 'https://api-web.nhle.com/v1'
PLAYER_ID_FILENAME = 'playerIDs.json'

We need to get all team abbreviations to look up each team's roster. We could use https://api.nhle.com/stats/rest/en/team but as of right now we don't want to include the teams that don't exist anymore. Instead we will use the current season standings to get the team abbreviations

In [31]:
def fetch_team_abbrevs():
    data = requests.get(f'{BASE_ENDPOINT}/standings/now').json()
    data = data['standings']
    team_abbrevs = [team['teamAbbrev']['default'] for team in data]
    return team_abbrevs
fetch_team_abbrevs()

['VAN',
 'BOS',
 'FLA',
 'DAL',
 'NYR',
 'VGK',
 'COL',
 'WPG',
 'CAR',
 'PHI',
 'EDM',
 'TBL',
 'TOR',
 'DET',
 'LAK',
 'STL',
 'NSH',
 'NYI',
 'CGY',
 'NJD',
 'WSH',
 'PIT',
 'SEA',
 'MIN',
 'ARI',
 'MTL',
 'BUF',
 'OTT',
 'CBJ',
 'ANA',
 'SJS',
 'CHI']

Get all team rosters by abbreviations:

In [32]:
def fetch_rosters():
    team_abbrevs = fetch_team_abbrevs()
    rosters = [requests.get(f'{BASE_ENDPOINT}/roster/{abbrev}/current').json() for abbrev in team_abbrevs]
    return rosters
fetch_rosters()

[{'forwards': [{'id': 8482496,
    'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/VAN/8482496.png',
    'firstName': {'default': 'Nils'},
    'lastName': {'default': 'Aman'},
    'sweaterNumber': 88,
    'positionCode': 'C',
    'shootsCatches': 'L',
    'heightInInches': 74,
    'weightInPounds': 179,
    'heightInCentimeters': 188,
    'weightInKilograms': 81,
    'birthDate': '2000-02-07',
    'birthCity': {'default': 'Avesta'},
    'birthCountry': 'SWE'},
   {'id': 8476927,
    'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/VAN/8476927.png',
    'firstName': {'default': 'Teddy', 'cs': 'Theodors', 'sk': 'Theodors'},
    'lastName': {'default': 'Blueger', 'cs': 'Blugers', 'sk': 'Blugers'},
    'sweaterNumber': 53,
    'positionCode': 'C',
    'shootsCatches': 'L',
    'heightInInches': 72,
    'weightInPounds': 185,
    'heightInCentimeters': 183,
    'weightInKilograms': 84,
    'birthDate': '1994-08-15',
    'birthCity': {'default': 'Riga', 'fi': 'Riika'},
    'birt

Pull each player's id in each roster to a list:

In [33]:
def get_player_ids():
    rosters = fetch_rosters()

    player_ids = dict()
    for roster_by_posn in rosters:
        roster = roster_by_posn['forwards'] + roster_by_posn['defensemen'] + roster_by_posn['goalies']
        for player in roster:
            first_name = player['firstName']['default']
            last_name = player['lastName']['default']
            key = f'{first_name} {last_name}'
            
            if key in player_ids:
                # extract team abbr from headshot link
                headshot_ep = player['headshot']
                player_team_abbrev = headshot_ep.split('/')[-2]
                key += f' ({player_team_abbrev})'
            
            player_id = player['id']
            player_ids[key] = player_id

    return player_ids

get_player_ids()

{'Nils Aman': 8482496,
 'Teddy Blueger': 8476927,
 'Brock Boeser': 8478444,
 'Phillip Di Giuseppe': 8476858,
 'Conor Garland': 8478856,
 'Nils Hoglander': 8481535,
 'Dakota Joshua': 8478057,
 'Sam Lafferty': 8478043,
 'Elias Lindholm': 8477496,
 'Ilya Mikheyev': 8481624,
 'J.T. Miller': 8476468,
 'Elias Pettersson': 8480012,
 'Pius Suter': 8480459,
 'Guillaume Brisebois': 8478465,
 'Ian Cole': 8474013,
 'Mark Friedman': 8478017,
 'Filip Hronek': 8479425,
 'Quinn Hughes': 8480800,
 'Noah Juulsen': 8478454,
 'Tyler Myers': 8474574,
 'Tucker Poolman': 8477359,
 'Carson Soucy': 8477369,
 'Jett Woo': 8480808,
 'Nikita Zadorov': 8477507,
 'Thatcher Demko': 8477967,
 'Casey DeSmith': 8479193,
 'Jesper Boqvist': 8480003,
 'Charlie Coyle': 8475745,
 'Jake DeBrusk': 8478498,
 'Trent Frederic': 8479365,
 'Morgan Geekie': 8479987,
 'Danton Heinen': 8478046,
 'Jakub Lauko': 8480880,
 'Milan Lucic': 8473473,
 'Brad Marchand': 8473419,
 'David Pastrnak': 8477956,
 'Matthew Poitras': 8483505,
 'Anthon

Finally, write to file:

In [34]:
def write_ids():
    player_ids = get_player_ids()
    jobj = json.dumps(player_ids)
    with open(PLAYER_ID_FILENAME, 'w') as outfile:
        outfile.write(jobj)